<a href="https://www.kaggle.com/code/keerthi4701/mnist-degit-classification-cnn?scriptVersionId=138188444" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Reading the dataset
df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df.head()

In [ ]:
#extracting the target column
target = df.pop('label')
target

In [ ]:
display(df.head())

print('\n\n  df.describe()')
df.describe()

In [ ]:
def preprocess(data):
    
    data = (data/255).astype('float32')   #scaling the data to 0-1 from 0-255
    
    X = data.values.reshape(-1,28,28,1)  # converting 784 columns to 28x28x1 array
    return X
    

In [ ]:
X = preprocess(df)

# plotting some images
fig, axes = plt.subplots(5,5, figsize=(10,10))
for i,ax in enumerate(axes.flat):
    ax.imshow(X[i])

In [ ]:
random = np.random.randint(100)

print("label :",target[random])  #checking weather target matches with image array
plt.imshow(X[random])

In [ ]:
#Converting target column to Categorical 2D array 
y = keras.utils.to_categorical(target, 10)
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

In [ ]:
model = Sequential()
model.add(Conv2D(20, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(Conv2D(20, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=100,
          epochs=4,
          validation_split = 0.2)

In [ ]:
def predict(data):
    predictions = model.predict(data)
    predictions = (predictions >= 0.5).astype(int)
    predictions = pd.Series(tf.math.argmax(predictions, axis=1))
    return predictions
    

In [ ]:
pred = predict(X_test)
pred

In [ ]:
random = np.random.randint(100)

print("label :",pred[random])  #checking weather target matches with image array
plt.imshow(X_test[random])

# Kaggle submission

In [ ]:
#kaggle submission
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
df_test=preprocess(test)
y_pred = model.predict(df_test)

In [ ]:
y_pred = (y_pred >= 0.5).astype(int)
label = tf.math.argmax(y_pred, axis=1)
output = pd.DataFrame({"ImageId":test.index +1, "Label":label})
output

In [ ]:
output.to_csv("submission.csv", index=False)